# Download forcing
Downloads ERA5 forcing data. Time period to download is based on availability of both flow and forcing data.
- Flow: catchment-dependent
- ERA5: 1940-current
- EM-Earth: 1950-2019


We want to extract data on a per-month basis, particularly for MARS request (i.e. tape archive) because this maximizes the amount of information we extract from a single tape. This prevents unnecessary tape switching. This hopefully strikes a good balance between needs.

See:
- https://confluence.ecmwf.int/display/UDOC/Retrieve#Retrieve-Datacollocation

In [1]:
import sys
from datetime import datetime, timedelta
import pandas as pd
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path = cs.read_from_config(config_file,'data_path')

# CAMELS-spat metadata
cs_meta_path = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name = cs.read_from_config(config_file,'cs_meta_name')
cs_unusable_name = cs.read_from_config(config_file,'cs_unusable_name')

# Basin folder
cs_basin_folder = cs.read_from_config(config_file, 'cs_basin_path')
basins_path = Path(data_path) / cs_basin_folder

### Data loading

In [4]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

In [5]:
# Open list of unusable stations; Enforce reading IDs as string to keep leading 0's
cs_unusable = pd.read_csv(cs_meta_path / cs_unusable_name, dtype={'Station_id': object}) 

## Processing

In [ ]:
# Per basin:
# 1. Determine download coordinates (code from cwarhm)
# 2. Determine download period (from cs_meta)
# 3. Download ERA5 
# 4. Download deterministic EM-Earth

In [83]:
for ix,row in cs_meta.iterrows():

    # TESTING: run 1 station only
    if ix != 0: continue 
    
    # Get shapefile path to determine download coordinates, and forcing destination path
    basin_id, shp_lump_path, _, _, _ = cs.prepare_delineation_outputs(cs_meta, ix, Path(data_path)/cs_basin_folder)
    raw_fold, _, _ = prepare_forcing_outputs(cs_meta, ix, Path(data_path)/cs_basin_folder) # Returns folders only, not file names
    print('--- Now running basin {}. {}'.format(ix, basin_id))
    
    # From shapefile, get bounding coordinates. Then determine download coordinates from those
    bounds = find_shapefile_bounds(shp_lump_path)
    coords_era5, _, _ = find_download_coords_from_bounds(bounds, target='ERA5')
    
    # Check if we need to run downloads for this station at all
    missing = flow_obs_unavailable(cs_unusable, row.Country, row.Station_id)
    if 'iv' in missing and 'dv' in missing: 
        continue # with next station, because we have no observations at all for this station

    # From meta-data, get download period
    times_flow = find_flow_obs_times_from_metadata(row, missing)
    times_era5 = round_flow_obs_to_days(times_flow)
    start_date = datetime.strptime(times_era5[0], '%Y-%m-%d')
    final_date = datetime.strptime(times_era5[1], '%Y-%m-%d')
    
    print(f'    Basin coordinates:         {bounds}')
    print(f'    ERA5 download coordinates: [{coords_era5[0]}]')
    print(f'    Flow obs unavailable:      {missing}')
    print(f'    Download times:            {times_era5}')

    # Convert start and end dates into two lists of start and end dates, that we'll iterate over
    final_date = final_date.replace(year=1950) # TESTING
    start_list,end_list = convert_start_and_end_dates_to_era5_download_lists(start_date,final_date)
    
    # Download data into destination folder
    download_era5_time_invariant_data_to_netcdf(coords_era5, raw_fold/'ERA5_2023-01-01_invariants.nc')

    for start,end in zip(start_list,end_list):

        # Convert to relevant
        yyyy_mm = start.strftime('%Y-%m') # filename
        start   = start.strftime('%Y-%m-%d') # yyyy-mm-dd for use with cdsapi
        end     = end.strftime('%Y-%m-%d')

        # Get the data
        download_era5_surface_level_data_to_netcdf(coords_era5, start, end, raw_fold/f'ERA5_{yyyy_mm}_surface_variables.nc')
        download_era5_pressure_level_data_to_netcdf(coords_era5, start, end, raw_fold/f'ERA5_{yyyy_mm}_pressure_variables.nc')

    print('--- Downloads for basin {} complete.'.format(ix, basin_id))

Now running basin 0. CAN_01AD002
    Basin coordinates:         [-70.43208333  45.98541667 -68.07125     47.83791667]
    ERA5 download coordinates: [4]
    Flow obs unavailable:      ['iv']
    Download times:            ['1950-01-01', '2020-12-31']


2023-08-20 17:30:11,641 INFO Welcome to the CDS
2023-08-20 17:30:11,641 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-20 17:30:11,863 INFO Request is queued
2023-08-20 17:30:13,077 INFO Request is running
2023-08-20 17:34:32,804 INFO Request is completed
2023-08-20 17:34:32,804 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data8/adaptor.mars.internal-1692574456.6285071-22756-2-d2b6cef2-d718-4ade-a8f1-6a6e736be074.nc to C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-01_surface_variables.nc (900.8K)
2023-08-20 17:34:35,147 INFO Download rate 384.4K/s


Successfully downloaded C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-01_surface_variables.nc


2023-08-20 17:34:35,555 INFO Welcome to the CDS
2023-08-20 17:34:35,555 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-complete
2023-08-20 17:34:35,816 INFO Request is queued
2023-08-20 17:34:37,206 INFO Request is running
2023-08-20 17:38:56,928 INFO Request is completed
2023-08-20 17:38:56,929 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data7/adaptor.mars.external-1692574640.5989888-7456-4-997c4416-87d1-4188-94fc-bdb639589698.nc to C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-01_pressure_variables.nc (516.4K)
2023-08-20 17:38:58,990 INFO Download rate 250.6K/s
2023-08-20 17:38:59,425 INFO Welcome to the CDS


Successfully downloaded C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-01_pressure_variables.nc


...

Successfully downloaded C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-11_pressure_variables.nc


2023-08-20 19:01:28,323 INFO Welcome to the CDS
2023-08-20 19:01:28,324 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-20 19:01:28,544 INFO Request is queued
2023-08-20 19:01:29,751 INFO Request is running
2023-08-20 19:07:50,036 INFO Request is completed
2023-08-20 19:07:50,036 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data8/adaptor.mars.internal-1692579940.3982298-5004-3-222874f1-0897-4d20-a02d-8cedd3fc6089.nc to C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-12_surface_variables.nc (900.8K)
2023-08-20 19:07:58,122 INFO Download rate 111.4K/s


Successfully downloaded C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-12_surface_variables.nc


2023-08-20 19:07:58,538 INFO Welcome to the CDS
2023-08-20 19:07:58,538 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-complete
2023-08-20 19:07:58,757 INFO Request is queued
2023-08-20 19:08:04,383 INFO Request is running
2023-08-20 19:12:19,709 INFO Request is completed
2023-08-20 19:12:19,711 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data9/adaptor.mars.external-1692580244.1355445-13334-1-79962571-50d2-4233-9bc2-9f99b51e2e8a.nc to C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-12_pressure_variables.nc (516.4K)
2023-08-20 19:12:23,763 INFO Download rate 127.5K/s


Successfully downloaded C:\Globus endpoint\CAMELS_spat\camels-spat-data\basin_data\CAN_01AD002\forcing\raw\ERA5_1950-12_pressure_variables.nc
    Downloads complete.


### Functions

In [38]:
from datetime import datetime, timedelta
import geopandas as gpd
import math
import cdsapi
import os

In [7]:
def prepare_forcing_outputs(df,i,data_path):
    
    '''Prepares output folders for lumped and distributed forcing downloads outcomes'''
    
    from pathlib import Path
    
    # Get identifiers
    country = df.iloc[i].Country
    basin_id = df.iloc[i].Station_id
    full_id = country + '_' + basin_id
    
    # Construct the paths
    main_folder = Path(data_path) / 'basin_data' / (country + '_' + basin_id) / 'forcing' 
    lump_folder = main_folder / 'lumped'
    dist_folder = main_folder / 'distributed'
    raw_folder  = main_folder / 'raw'
    
    # Make the paths
    lump_folder.mkdir(parents=True, exist_ok=True)
    dist_folder.mkdir(parents=True, exist_ok=True)
    raw_folder.mkdir(parents=True, exist_ok=True)
   
    return raw_folder, lump_folder, dist_folder

In [80]:
def download_era5_surface_level_data_to_netcdf(coordinates,date_s,date_e,file):
    
    '''Downloads specified ERA5 surface level parameters for a specified bounding box'''
    # Pressure level variables: 

    if not os.path.isfile(file):
        # Make sure the connection is re-tried if it fails
        retries_max = 1
        retries_cur = 1
        while retries_cur <= retries_max:
            try:
    
                # connect to Copernicus (requires .cdsapirc file in $HOME)
                c = cdsapi.Client()
            
                # specify and retrieve data
                c.retrieve('reanalysis-era5-single-levels', { # do not change this!
                           'product_type': 'reanalysis',
                           'format'      : 'netcdf',
                           'variable'    : [
                               'mean_surface_downward_long_wave_radiation_flux',
                               'mean_surface_net_long_wave_radiation_flux',
                               'mean_surface_downward_short_wave_radiation_flux',
                               'mean_surface_net_short_wave_radiation_flux',
                               'mean_total_precipitation_rate',
                               'surface_pressure',
                               'mean_potential_evaporation_rate',
                           ],
                           'date': f'{date_s}/{date_e}', # 'yyyy-mm-dd'
                           'time': '00/to/23/by/1',
                           'area': coordinates, # expected as [lat_max, lon_min, lat_min, lon_max], e.g. [51.75/-116.5/51.0/-115.5]
                           'grid': '0.25/0.25', # Latitude/longitude grid: east-west (longitude) and north-south resolution (latitude).
                    },
                    file) # file path and name

                # track progress
                print('Successfully downloaded ' + str(file))

            except:
                print('Error downloading ' + str(file) + ' on try ' + str(retries_cur))
                retries_cur += 1
                continue
            else:
                break
    return

In [81]:
def download_era5_pressure_level_data_to_netcdf(coordinates,date_s,date_e,file):
    
    '''Downloads specified ERA5 pressure level parameters for a specified bounding box'''
    # Pressure level variables: https://confluence.ecmwf.int/pages/viewpage.action?pageId=82870405#ERA5:datadocumentation-Table9
    # MARS requests: https://confluence.ecmwf.int/display/UDOC/HRES%3A+Atmospheric+%28oper%29%2C+Model+level+%28ml%29%2C+Forecast+%28fc%29%3A+Guidelines+to+write+efficient+MARS+requests
    
    if not os.path.isfile(file):
        # Make sure the connection is re-tried if it fails
        retries_max = 1
        retries_cur = 1
        while retries_cur <= retries_max:
            try:
    
                # connect to Copernicus (requires .cdsapirc file in $HOME)
                c = cdsapi.Client()
            
                # specify and retrieve data
                c.retrieve('reanalysis-era5-complete', {    # do not change this!
                           'class'   : 'ea',
                           'expver'  : '1',
                           'stream'  : 'oper',
                           'type'    : 'an',
                           'levtype' : 'ml',
                           'levelist': '137',
                           'param'   : '130/131/132/133', # i.e., Temperature, U and V wind, specific humidity
                           'date'    : f'{date_s}/to/{date_e}', # 'yyyy-mm-dd'
                           'time'    : '00/to/23/by/1', 
                           'area'    : coordinates, # expected as [lat_max, lon_min, lat_min, lon_max], e.g. [51.75/-116.5/51.0/-115.5]
                           'grid'    : '0.25/0.25', # Latitude/longitude grid: east-west (longitude) and north-south resolution (latitude).
                           'format'  : 'netcdf',
                    }, file)

                # track progress
                print('Successfully downloaded ' + str(file))

            except:
                print('Error downloading ' + str(file) + ' on try ' + str(retries_cur))
                retries_cur += 1
                continue
            else:
                break
    return

In [10]:
def download_era5_time_invariant_data_to_netcdf(coordinates,file):
    
    '''Downloads all ERA5 time-invariant parameters for a specified bounding box'''
    # Time-invariants: https://confluence.ecmwf.int/pages/viewpage.action?pageId=82870405#ERA5:datadocumentation-Table1

    if not os.path.isfile(file):
        # Make sure the connection is re-tried if it fails
        retries_max = 1
        retries_cur = 1
        while retries_cur <= retries_max:
            try:
    
                # connect to Copernicus (requires .cdsapirc file in $HOME)
                c = cdsapi.Client()
            
                # specify and retrieve data
                c.retrieve('reanalysis-era5-complete', {    # do not change this!
                           'stream' : 'oper',
                           'levtype': 'sf',
                           'param'  : '26/228007/27/28/29/30/43/74/129/160/161/162/163/172', # i.e., all time-invariant values
                           'date'   : '2023-01-01', # arbitrary date
                           'time'   : '00', # time-invariant data; no need to get more than a single time step
                           'area'   : coordinates, # expected as [lat_max, lon_min, lat_min, lon_max], e.g. [51.75/-116.5/51.0/-115.5]
                           'grid'   : '0.25/0.25', # Latitude/longitude grid: east-west (longitude) and north-south resolution (latitude).
                           'format' : 'netcdf',
                    }, file)

                # track progress
                print('Successfully downloaded ' + str(file))

            except:
                print('Error downloading ' + str(file) + ' on try ' + str(retries_cur))
                retries_cur += 1
                continue
            else:
                break
    return

In [79]:
def convert_start_and_end_dates_to_era5_download_lists(start,end):

    '''Takes two datetime.datetime(y,m,d,h,min) objects and returns two lists with start and end dates for ERA5 downloads at monthly intervals'''

    # Initiate the date we're current working with
    cur = start 
    
    # Initiate the lists
    start_l = []
    end_l = []
    
    # Loop over the dates, until we have the end date
    while cur < end:
        
        # Add to start list
        start_l.append(cur)
        
        # Figure out the index of the next month and if the year changes:
        tmp = cur + timedelta(days=31) # Add 31 days to current date to ensure we're in the next month, might also switch the year
        next_month = tmp.month #  Extract 'month' from this object
        next_year = tmp.year # Extract the year too. If we ticked over into a new year we need to track this, otherwise we never increment the year

        # Create the end-of-month date
        cur = cur.replace(year=next_year, month=next_month) - timedelta(days=1)
        
        # Ensure this does not step over our end date
        if cur >= end:
            cur = end

        # Add to end list
        end_l.append(cur)

        # Add 1 day to create the new start-of-month date
        cur = cur + timedelta(days=1)
        
    return start_l,end_l

In [11]:
def convert_to_date_only(time_string, original_format="%Y-%m-%d %H:%M:%S", target_format="%Y-%m-%d"):
    try:
        datetime_obj = datetime.strptime(time_string, original_format)
        date_only_string = datetime_obj.strftime(target_format)
        return date_only_string
    except ValueError:
        return None

In [12]:
def round_flow_obs_to_days(times):
    
    '''Takes two times ([time1,time2]) in 'YYYY-MM-DD hh:mm:ss' and rounds to days '''
    
    return [convert_to_date_only(times[0]),
            convert_to_date_only(times[1])]

In [13]:
def flow_obs_unavailable(df,country,station):
    
    '''Checks in the "unusable" dataframe if iv, dv or both are unavailable for a station'''
    
    missing = []
    for ix,row in df.iterrows():
        if row.Country != country:
            continue
        if row.Station_id == station:
            missing.append(row.Missing)
    
    return missing

In [14]:
def find_flow_obs_times_from_metadata(row,missing):
    
    '''Finds required data start and end times from flow observations in meta-data file'''
    
    # Start and end dates come as 'YYYY-MM-DD hh:mm:ss' strings so we can directly compare them
    # Source: https://stackoverflow.com/a/54987418
    
    # Shorthands
    iv_s = row.iv_flow_obs_availability_start
    iv_e = row.iv_flow_obs_availability_end
    dv_s = row.dv_flow_obs_availability_start
    dv_e = row.dv_flow_obs_availability_end
    
    # Missing data cases
    if ('iv' in missing) and ('dv' in missing):
        return []
    
    elif 'iv' in missing:
        times = [dv_s,dv_e]
        for time in times: 
            assert is_valid_date_format(time), f'{time} not in expected format'
        return [dv_s,dv_e]
    
    elif 'dv' in missing:
        times = [iv_s,iv_e]
        for time in times: 
            assert is_valid_date_format(time), f'{time} not in expected format'
        return [iv_s,iv_e]
    
    else:
        times = [iv_s,iv_e,dv_s,dv_e]
        for time in times: 
            assert is_valid_date_format(time), f'{time} not in expected format'
    return [min(iv_s,dv_s),max(iv_e,dv_e)]

In [15]:
def is_valid_date_format(date_string, date_format='%Y-%m-%d %H:%M:%S'):
    try:
        datetime.strptime(date_string, date_format)
        return True
    except ValueError:
        return False

In [16]:
# Find shapefile bounding box
def find_shapefile_bounds(path):
    
    # Modified from: https://github.com/CH-Earth/CWARHM/blob/main/0_tools/ERA5_find_download_coordinates_from_shapefile.ipynb
    shp = gpd.read_file(path)
    
    return shp.total_bounds

In [17]:
def find_download_coords_from_bounds(coords, target='ERA5'):
    
    '''
    Determines download coordinates from shapefile bounds for a given data set.
    Assumes coodinates are an array: [lon_min, lat_min, lon_max, lat_max] (bottom-left, top-right).
    Returns separate lat and lon vectors.
    '''

    # Source: https://github.com/CH-Earth/CWARHM/blob/main/3a_forcing/1a_download_forcing/download_ERA5_pressureLevel_annual.ipynb   
    
    # Extract values
    lon = [coords[0],coords[2]]
    lat = [coords[1],coords[3]]
    
    if target == 'ERA5':
        
        # Round to ERA5 0.25 degree resolution
        rounded_lon = [math.floor(lon[0]*4)/4, math.ceil(lon[1]*4)/4]
        rounded_lat = [math.floor(lat[0]*4)/4, math.ceil(lat[1]*4)/4]

        # Find if we are still in the representative area of a different ERA5 grid cell
        if lat[0] > rounded_lat[0]+0.125:
            rounded_lat[0] += 0.25
        if lon[0] > rounded_lon[0]+0.125:
            rounded_lon[0] += 0.25
        if lat[1] < rounded_lat[1]-0.125:
            rounded_lat[1] -= 0.25
        if lon[1] < rounded_lon[1]-0.125:
            rounded_lon[1] -= 0.25
    
        # Make a download string ready for ERA5 (cdsapi) format
        dl_string = '{}/{}/{}/{}'.format(rounded_lat[1],rounded_lon[0],rounded_lat[0],rounded_lon[1])
    
    return dl_string, rounded_lat, rounded_lon